In [31]:
import os
from sqlalchemy import create_engine


# Construct the connection URL
db_url = f"sqlite:///{os.path.abspath('/home/muregz/data_analysis/song_sales/chinook.db')}"

# Set the DATABASE_URL environment variable
os.environ['DATABASE_URL'] = db_url

# Create a database engine
engine = create_engine(os.environ['DATABASE_URL'])


In [32]:
#configuring mysql into jupyter notebook
%load_ext sql
%config SqlMagic.autopandas=True

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


**1.Which tracks generated the most revenue? which album? which genre?**

In [38]:
%%sql
SELECT tracks.Name AS `Song`, albums.Title AS `Album`, genres.Name 
AS `Genre`, SUM(invoice_items.UnitPrice * invoice_items.Quantity) AS Revenue
FROM tracks
JOIN albums ON tracks.AlbumId = albums.AlbumId
JOIN genres ON tracks.GenreId = genres.GenreId
JOIN invoice_items ON tracks.TrackId = invoice_items.TrackId
GROUP BY tracks.TrackId
ORDER BY revenue DESC
LIMIT 5;


 * sqlite:////home/muregz/data_analysis/song_sales/chinook.db
Done.


,Song,Album,Genre,Revenue
0,The Woman King,"Battlestar Galactica, Season 3",Science Fiction,3.98
1,The Fix,"Heroes, Season 1",Drama,3.98
2,Walkabout,"Lost, Season 1",TV Shows,3.98
3,Hot Girl,"The Office, Season 1",TV Shows,3.98
4,Gay Witch Hunt,"The Office, Season 3",TV Shows,3.98


**2.Which tracks appeared in the most playlists?How many playlists did they appear in?**

In [37]:
%%sql
SELECT tracks.Name, COUNT(*) AS `Number of Appearances`
FROM playlist_track
JOIN tracks
ON playlist_track.TrackId = tracks.TrackId
GROUP BY playlist_track.TrackId
ORDER BY COUNT(*) DESC
LIMIT 10;

 * sqlite:////home/muregz/data_analysis/song_sales/chinook.db
Done.


,Name,Number of Appearances
0,Intoitus: Adorate Deum,5
1,"Miserere mei, Deus",5
2,"Aria Mit 30 Veränderungen, BWV 988 ""Goldberg V...",5
3,"Suite for Solo Cello No. 1 in G Major, BWV 100...",5
4,"The Messiah: Behold, I Tell You a Mystery... T...",5
5,Solomon HWV 67: The Arrival of the Queen of Sheba,5
6,Symphony No.5 in C Minor: I. Allegro con brio,5
7,Ave Maria,5
8,"Nabucco: Chorus, ""Va, Pensiero, Sull'ali Dorate""",5
9,Die Walküre: The Ride of the Valkyries,5


**3.Which countries have the highest sales revenue? What percentage of total revenue does each country make up?**

In [ ]:
%%sql
SELECT invoices.BillingCountry AS `Country`, SUM(invoices.Total) AS Revenue,
ROUND((SUM(invoices.Total) * 100)/SUM(SUM(invoices.Total)) OVER(), 2) AS `Percentage of Total Revenue`
FROM invoices
GROUP BY invoices.BillingCountry
ORDER BY Revenue DESC
LIMIT 10;

**4.How many customers did each employee support, what is the average revenue for each sale, and what is their total sale?**

In [46]:
%%sql
SELECT employees.FirstName, employees.LastName AS `Employee Name`,
SUM(invoices.Total) AS Revenue
FROM employees
JOIN customers ON employees.EmployeeId = customers.SupportRepId
JOIN invoices ON customers.SupportRepId = invoices.CustomerId
GROUP BY `Employee Name`
ORDER BY Revenue DESC
LIMIT 10;

 * sqlite:////home/muregz/data_analysis/song_sales/chinook.db
Done.


,FirstName,Employee Name,Revenue
0,Jane,Peacock,832.02
1,Margaret,Park,792.40
2,Steve,Johnson,731.16
